## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

Материалы.<br>
У вас лекционный ноутбук в архиве он с картинками и презентация тоже есть в архиве

**Задание**

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими.

Постарайтесь выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
  1. построить свёрточные архитектуры
  2. построить различные архитектуры с RNN
  3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
  4. сдлать выводы что получилось лучше


## Решение. 

Загрузим датасет и выполним предобработку данных.

In [1]:
# !pip install stop_words

In [2]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from tqdm import tqdm

from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py

tqdm.pandas()

/home/sergey/anaconda3/envs/deeppavlov_env/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")

In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

# Теперь повторим это для всех записей.
def replase_words(text,dict_): 
    output = ''
    for word in text.split(' '): # TODO не будем делить текст на части будем искать подстроку в строке. это касается только смайликов.
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

def preprocess_text(txt):
    txt = str(txt)
    txt = re.sub("[\,]","",txt)
    txt = re.sub("@[\w]*","",txt)
    # txt = re.sub("RT","",txt)
    # Заменим эмодзи на соответствующие им слова.
    txt = replase_words(txt, emoticon_dict)
     # Заменим сокращения на их полные формы
    txt = replase_words(txt, apostrophe_dict)
    txt = replase_words(txt, short_word_dict)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)#[w for w in txt if len(w)>1])

In [6]:
df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)

100%|██████████| 22683/22683 [00:45<00:00, 493.93it/s]


In [7]:
df_train.head()

,id,text,class
0,0,уезжаааааааать ❤ тожена хотеть уезжать,0
1,1,rt ребята девчата кино любовь завтра вотэтолюбовь,1
2,2,rt ктоненавидеть пробка ретвит rt,0
3,3,rt хотеться котлета покиевск запретный плод happy,1
4,4,босапоп есбосан бояться мороз,1


In [8]:
import numpy as np
# import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking,MaxPooling1D, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
# from tensorflow.keras.objectives import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  

In [9]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values

In [10]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [11]:
word_count, training_length

(188809, 28)

In [12]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [13]:
results = {
    "NN":[],
    "loss":[],
    "accuracy":[]
}

In [14]:
early_stopping=EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=3)

### CNN

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Train on 163320 samples, validate on 18147 samples
Epoch 1/5
163320/163320 [==============================] - 25s 153us/sample - loss: 0.5008 - acc: 0.7287 - val_loss: 0.4304 - val_acc: 0.7813
Epoch 2/5
163320/163320 [==============================] - 25s 151us/sample - loss: 0.3148 - acc: 0.8622 - val_loss: 0.4607 - val_acc: 0.7737


In [17]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
# print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("CNN")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Test score: 0.474356822238231
Test accuracy: 0.7707975


## SimpleRNN

In [18]:

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Train on 163320 samples, validate on 18147 samples
Epoch 1/10
163320/163320 [==============================] - 26s 158us/sample - loss: 0.4958 - acc: 0.7352 - val_loss: 0.4327 - val_acc: 0.7794
Epoch 2/10
163320/163320 [==============================] - 26s 157us/sample - loss: 0.2884 - acc: 0.8768 - val_loss: 0.4824 - val_acc: 0.7692


In [20]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("SimpleRNN")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Test score: 0.47969025303955587
Test accuracy: 0.76850504


## LSTM

In [21]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Train on 163320 samples, validate on 18147 samples
Epoch 1/10
163320/163320 [==============================] - 40s 244us/sample - loss: 0.4944 - acc: 0.7456 - val_loss: 0.4351 - val_acc: 0.7787
Epoch 2/10
163320/163320 [==============================] - 39s 240us/sample - loss: 0.3167 - acc: 0.8640 - val_loss: 0.4694 - val_acc: 0.7711


In [23]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("LSTM")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Test score: 0.4719575312393754
Test accuracy: 0.7721201


## GRU

In [24]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Train on 163320 samples, validate on 18147 samples
Epoch 1/10
163320/163320 [==============================] - 36s 218us/sample - loss: 0.4905 - acc: 0.7459 - val_loss: 0.4366 - val_acc: 0.7768
Epoch 2/10
163320/163320 [==============================] - 36s 218us/sample - loss: 0.3113 - acc: 0.8650 - val_loss: 0.4670 - val_acc: 0.7698


In [26]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("GRU")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Test score: 0.46716090671411664
Test accuracy: 0.7682405


## CNN+RNN

In [27]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(32, 2))
model.add(Activation("relu"))

model.add(Conv1D(16, 2))
model.add(Activation("relu"))
model.add(MaxPooling1D(1))

model.add(LSTM(16,return_sequences=True))

model.add(LSTM(16))

model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
                    
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("CNN+RNN")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Train on 163320 samples, validate on 18147 samples
Epoch 1/10
163320/163320 [==============================] - 31s 190us/sample - loss: 0.5119 - acc: 0.7212 - val_loss: 0.4338 - val_acc: 0.7818
Epoch 2/10
163320/163320 [==============================] - 30s 184us/sample - loss: 0.3099 - acc: 0.8636 - val_loss: 0.4615 - val_acc: 0.7719
Test score: 0.46594160206470414
Test accuracy: 0.77194375


## RNN+CNN

In [33]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))

model.add(Masking(mask_value=0.0))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32,return_sequences=True))

model.add(Conv1D(32, 3))
model.add(Activation("relu"))

model.add(MaxPooling1D(2))
model.add(Activation("relu"))

model.add(Conv1D(16, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
                    
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results['NN'].append("RNN+CNN")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Train on 163320 samples, validate on 18147 samples
Epoch 1/10
163320/163320 [==============================] - 111s 681us/sample - loss: 0.5264 - acc: 0.7075 - val_loss: 0.4364 - val_acc: 0.7807
Epoch 2/10
163320/163320 [==============================] - 105s 645us/sample - loss: 0.3252 - acc: 0.8563 - val_loss: 0.4596 - val_acc: 0.7741
Test score: 0.46472977928681614
Test accuracy: 0.7755147


In [35]:
pd.DataFrame(results)

,NN,loss,accuracy
0,CNN,0.474357,0.770797
1,SimpleRNN,0.479690,0.768505
2,LSTM,0.471958,0.772120
3,GRU,0.467161,0.768241
4,CNN+RNN,0.465942,0.771944
5,RNN+CNN,0.464730,0.775515


### Резюме. 

Проверил различные варианты нейронных сетей. Экспериментрировал с числом слоев CNN и RNN. Качество от этого не сильно изменялось. По итогу, все рассмотренные архитектуры показали примерно оденаковый результат.  